In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
plt.style.use("ggplot")

In [2]:
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.ensemble import RandomForestClassifier

# Preprocessing

In [4]:
data_raw = pd.read_csv("monthlywithmomentum.csv", index_col=0)
y_raw = pd.read_csv("y.csv")

In [5]:
y_raw.head()

,date,TICKER,Y
0,19860331,NOAX,1
1,19860331,ALD,1
2,19860331,AC,1
3,19860331,BU,0
4,19860331,BG,0


In [6]:
data_raw.head()

,PERMNO,date,SHRCD,TICKER,COMNAM,CUSIP,PRC,VOL,RET,CFACPR,...,momentum63,momentum84,momentum105,momentum126,momentum147,momentum168,momentum189,momentum210,momentum231,momentum252
0,10062,19860331,11.0,NOAX,NORTHEAST OHIO AXLE INC,29409K60,-10.250,78.0,-0.012048,0.143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10062,19860430,11.0,NOAX,NORTHEAST OHIO AXLE INC,29409K60,-13.875,508.0,0.353659,0.143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10062,19860530,11.0,NOAX,N E O A X INC,29409K60,13.750,335.0,-0.009009,0.143,...,-0.231741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10062,19860630,11.0,NOAX,N E O A X INC,29409K60,15.250,366.0,0.109091,0.143,...,-0.050081,-0.141722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10062,19860731,11.0,NOAX,N E O A X INC,29409K60,13.250,547.0,-0.131148,0.143,...,0.553784,0.103260,0.007698,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# drop na
data_raw = data_raw.dropna()
y_raw = y_raw.loc[data_raw.index]


,date,TICKER,Y,dateDT
5282,1965-12-31,AA,0,1965-12-31
5283,1965-12-31,AC,1,1965-12-31
5284,1965-12-31,ACD,0,1965-12-31
5285,1965-12-31,AGO,0,1965-12-31
5286,1965-12-31,ALA,1,1965-12-31
5287,1965-12-31,AMR,1,1965-12-31
5288,1965-12-31,APY,0,1965-12-31
5289,1965-12-31,ASA,1,1965-12-31
5290,1965-12-31,AUM,0,1965-12-31
5291,1965-12-31,AZP,1,1965-12-31


In [8]:
if not isinstance(data_raw['date'][0], str):
    data_raw["date"] = data_raw["date"].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:])
data_raw["dateDT"] = pd.to_datetime(data_raw["date"])

if not isinstance(y_raw['date'][0], str):
    y_raw["date"] = y_raw["date"].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:])
y_raw["dateDT"] = pd.to_datetime(y_raw["date"])

In [9]:
data_raw = data_raw.sort_values(by=["dateDT", "TICKER"])
data_raw.index = range(data_raw.shape[0])

In [10]:
y_raw = y_raw.sort_values(by=["dateDT", "TICKER"])
y_raw.index = range(y_raw.shape[0])

In [11]:
data_raw["id"] = data_raw.index

In [12]:
# split train, val, test
train_val_split = int(data_raw.shape[0]*0.4)
val_test_split = int(data_raw.shape[0]*0.6)
data_train = data_raw.loc[data_raw["dateDT"] <= data_raw["dateDT"][train_val_split]]
train_cut = data_train.shape[0]
data_val = data_raw.iloc[train_cut:].loc[data_raw["dateDT"] <= data_raw["dateDT"][val_test_split]]
val_cut = train_cut + data_val.shape[0]
data_test = data_raw.iloc[val_cut:]

In [13]:
y_train = y_raw.iloc[:train_cut]
y_val = y_raw.iloc[train_cut:val_cut]
y_test = y_raw.iloc[val_cut:]

In [14]:
feature_all = ['PRC', 'VOL', 'RET']

# add momentum terms
for i in range(1,22):
    feature_all.append("momentum" + str(i))
for i in range(42,253,21):
    feature_all.append("momentum" + str(i))

feature_all

['PRC',
 'VOL',
 'RET',
 'momentum1',
 'momentum2',
 'momentum3',
 'momentum4',
 'momentum5',
 'momentum6',
 'momentum7',
 'momentum8',
 'momentum9',
 'momentum10',
 'momentum11',
 'momentum12',
 'momentum13',
 'momentum14',
 'momentum15',
 'momentum16',
 'momentum17',
 'momentum18',
 'momentum19',
 'momentum20',
 'momentum21',
 'momentum42',
 'momentum63',
 'momentum84',
 'momentum105',
 'momentum126',
 'momentum147',
 'momentum168',
 'momentum189',
 'momentum210',
 'momentum231',
 'momentum252']

Use the following

`data_train`, `y_train`

`data_val`, `y_val`

`data_test`, `y_test`

# Descriptive Analysis

In [15]:
y_raw.shape

(436627, 4)

In [16]:
data_raw.shape

(436626, 47)

# Basic Models

In [17]:
lasso1 = LassoCV()
lasso1.fit(data_train[feature_all], y_train['Y'])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').